# Code for the paper: 

## Efficient and Mobile Deep Learning Architectures for Fast Identification of BacterialStrains in Resource-Constrained Devices

### Architecture: EfficientNet B0
### Data: Original + Cross-Validation

#### Loading libraries

In [2]:
# Imports here
import os
import torch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import torch.nn.functional as F
import pandas as pd
import time
import numpy as np

from PIL import Image
from sklearn.metrics import f1_score, precision_score, recall_score, classification_report
from sklearn.model_selection import KFold
from torch import nn
from torch import optim
from torch.utils.data import SubsetRandomSampler
from torchvision import datasets, transforms, models
from pytorch_model_summary import summary

# Archs not in Pytorch
from efficientnet_pytorch import EfficientNet

# External functions
from scripts.utils import *

In [2]:
print(torch.__version__)

import sys
print(sys.version)

1.5.1
3.8.5 (default, Jul 28 2020, 12:59:40) 
[GCC 9.3.0]


## Data paths and hyperparameters

#### Hyperparameters and dataset details. 

In [3]:
# Dataset details
dataset_version = 'original' # original or augmented
img_shape = (224,224)
img_size = str(img_shape[0])+"x"+str(img_shape[1])

# Root directory of dataset
data_dir = '/home/yibbtstll/venvs/pytorch_gpu/CySDeepBacterial/Dataset/DIBaS/'

train_batch_size = 32
val_test_batch_size = 32
feature_extract = False
pretrained = True
h_epochs = 15
kfolds = 10

## Data preparation and loading

#### Defining transforms and creating dataloaders

In [4]:
# Define transforms for input data
training_transforms = transforms.Compose([transforms.Resize((224,224), Image.LANCZOS),
                                          transforms.ToTensor(),
                                          transforms.Normalize([0.485, 0.456, 0.406], 
                                                               [0.229, 0.224, 0.225])])

# TODO: Load the datasets with ImageFolder
total_set = datasets.ImageFolder(data_dir, transform=training_transforms)

# Defining folds
splits = KFold(n_splits = kfolds, shuffle = True, random_state = 42)

#### Visualizing the target classes in dataset

In [5]:
train_labels = {value : key for (key, value) in total_set.class_to_idx.items()}
    
print(len(train_labels)) 
print(train_labels)

32
{0: 'Acinetobacter.baumanii', 1: 'Actinomyces.israeli', 2: 'Bacteroides.fragilis', 3: 'Bifidobacterium.spp', 4: 'Clostridium.perfringens', 5: 'Enterococcus.faecalis', 6: 'Enterococcus.faecium', 7: 'Escherichia.coli', 8: 'Fusobacterium', 9: 'Lactobacillus.casei', 10: 'Lactobacillus.crispatus', 11: 'Lactobacillus.delbrueckii', 12: 'Lactobacillus.gasseri', 13: 'Lactobacillus.jehnsenii', 14: 'Lactobacillus.johnsonii', 15: 'Lactobacillus.paracasei', 16: 'Lactobacillus.plantarum', 17: 'Lactobacillus.reuteri', 18: 'Lactobacillus.rhamnosus', 19: 'Lactobacillus.salivarius', 20: 'Listeria.monocytogenes', 21: 'Micrococcus.spp', 22: 'Neisseria.gonorrhoeae', 23: 'Porfyromonas.gingivalis', 24: 'Propionibacterium.acnes', 25: 'Proteus', 26: 'Pseudomonas.aeruginosa', 27: 'Staphylococcus.aureus', 28: 'Staphylococcus.epidermidis', 29: 'Staphylococcus.saprophiticus', 30: 'Streptococcus.agalactiae', 31: 'Veionella'}


## Model definition and inicialization

#### Freezing pre-trained parameters, finetunning the classifier to output 32 classes.

In [6]:
# Freeze pretrained model parameters to avoid backpropogating through them
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        print("Setting grad to false.")
        for param in model.parameters():
            param.requires_grad = False
    
    return model

def get_device():
    # Model and criterion to GPU
    if torch.cuda.is_available():
        return 'cuda'
    else:
        return 'cpu'

In [3]:
EfficientNet.from_name('efficientnet-b0')

EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
  )
  (_bn0): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_bn1): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_se_reduce): Conv2dStaticSamePadding(
        32, 8, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        8, 32, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        32, 16, kernel_size=

In [7]:
def load_model():
    # Transfer Learning
    model = EfficientNet.from_pretrained('efficientnet-b0')
    
    # Mode
    model = set_parameter_requires_grad(model, feature_extract)
    
    # Fine tuning
    # Build custom classifier
    model._fc = nn.Linear(in_features=1280,
                        out_features=32)
    return model

def create_optimizer(model):
    # Parameters to update
    params_to_update = model.parameters()

    if feature_extract:
        params_to_update = []
        for param in model.parameters():
            if param.requires_grad == True:
                params_to_update.append(param)

    else:
        n_params = 0
        for param in model.parameters():
            if param.requires_grad == True:
                n_params += 1


    # Loss function and gradient descent

    criterion = nn.CrossEntropyLoss()

    optimizer = optim.Adam(params_to_update, 
                          lr=0.001, 
                          weight_decay=0.000004)
    
    return criterion.to(get_device()), model.to(get_device()), optimizer

## Training, validation and test functions

In [8]:
# Variables to store fold scores
train_acc = []
test_top1_acc = []
test_top5_acc = []
test_precision = []
test_recall = []
test_f1 = []
times = []

for fold, (train_idx, valid_idx) in enumerate(splits.split(total_set)):
    
    start_time = time.time()
    
    print('Fold : {}'.format(fold))
    
    # Train and val samplers
    train_sampler = SubsetRandomSampler(train_idx)
    print("Samples in training:", len(train_sampler))
    valid_sampler = SubsetRandomSampler(valid_idx)
    print("Samples in test:", len(valid_sampler))
    
    # Train and val loaders
    train_loader = torch.utils.data.DataLoader(
                      total_set, batch_size=train_batch_size, sampler=train_sampler)
    valid_loader = torch.utils.data.DataLoader(
                      total_set, batch_size=1, sampler=valid_sampler)
    
    device = get_device()
    
    criterion, model, optimizer = create_optimizer(load_model())
    
    # Training
    for epoch in range(h_epochs):
        
        model.train()
        running_loss = 0.0
        running_corrects = 0
        trunning_corrects = 0
        
        for inputs, labels in train_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            
            with torch.set_grad_enabled(True):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                
            running_loss += loss.item() * inputs.size(0)
            running_corrects += (preds == labels).sum()
            trunning_corrects += preds.size(0)
            

        epoch_loss = running_loss / trunning_corrects
        epoch_acc = (running_corrects.double()*100) / trunning_corrects
        train_acc.append(epoch_acc.item())
        
        print('\t\t Training: Epoch({}) - Loss: {:.4f}, Acc: {:.4f}'.format(epoch, epoch_loss, epoch_acc))
        
        # Validation
        
        model.eval()  
        
        vrunning_loss = 0.0
        vrunning_corrects = 0
        num_samples = 0
        
        for data, labels in valid_loader:
            
            data = data.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            
            with torch.no_grad():
                outputs = model(data)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)
                
            vrunning_loss += loss.item() * data.size(0)
            vrunning_corrects += (preds == labels).sum()
            num_samples += preds.size(0)
            
        vepoch_loss = vrunning_loss/num_samples
        vepoch_acc = (vrunning_corrects.double() * 100)/num_samples
        
        print('\t\t Validation({}) - Loss: {:.4f}, Acc: {:.4f}'.format(epoch, vepoch_loss, vepoch_acc))
    
    # Calculating and appending scores to this fold
    scores = get_scores(model, valid_loader, total_set)
    
    test_top1_acc.append(scores[0])
    test_top5_acc.append(scores[1])
    test_precision.append(scores[2])
    test_recall.append(scores[3])
    test_f1.append(scores[4])
    
    time_fold = time.time() - start_time
    times.append(time_fold)
    print("Total time per fold: %s seconds." %(time_fold))

Fold : 0
Samples in training: 602
Samples in test: 67
Loaded pretrained weights for efficientnet-b0
		 Training: Epoch(0) - Loss: 2.2111, Acc: 49.8339
		 Validation(0) - Loss: 3.1885, Acc: 17.9104
		 Training: Epoch(1) - Loss: 0.5192, Acc: 85.5482
		 Validation(1) - Loss: 3.2964, Acc: 13.4328
		 Training: Epoch(2) - Loss: 0.2709, Acc: 92.3588
		 Validation(2) - Loss: 3.4652, Acc: 26.8657
		 Training: Epoch(3) - Loss: 0.2271, Acc: 92.1927
		 Validation(3) - Loss: 1.8388, Acc: 53.7313
		 Training: Epoch(4) - Loss: 0.2147, Acc: 93.8538
		 Validation(4) - Loss: 1.2688, Acc: 59.7015
		 Training: Epoch(5) - Loss: 0.1398, Acc: 97.0100
		 Validation(5) - Loss: 0.7419, Acc: 76.1194
		 Training: Epoch(6) - Loss: 0.1435, Acc: 95.5150
		 Validation(6) - Loss: 1.1701, Acc: 74.6269
		 Training: Epoch(7) - Loss: 0.0743, Acc: 98.0066
		 Validation(7) - Loss: 0.7241, Acc: 83.5821
		 Training: Epoch(8) - Loss: 0.0730, Acc: 97.6744
		 Validation(8) - Loss: 0.4762, Acc: 89.5522
		 Training: Epoch(9) - Los

		 Training: Epoch(10) - Loss: 0.0946, Acc: 97.0100
		 Validation(10) - Loss: 0.6325, Acc: 85.0746
		 Training: Epoch(11) - Loss: 0.1309, Acc: 97.3422
		 Validation(11) - Loss: 0.6329, Acc: 85.0746
		 Training: Epoch(12) - Loss: 0.0829, Acc: 98.1728
		 Validation(12) - Loss: 0.4727, Acc: 83.5821
		 Training: Epoch(13) - Loss: 0.0879, Acc: 97.1761
		 Validation(13) - Loss: 1.2176, Acc: 80.5970
		 Training: Epoch(14) - Loss: 0.0540, Acc: 98.8372
		 Validation(14) - Loss: 0.4548, Acc: 86.5672
Finished.
Top-1 Accuracy:  0.8656716417910447
Top-5 Accuracy:  0.9850746268656716
Weighted Precision 0.8614427860696516
Weighted Recall 0.8656716417910447
Weighted F1 0.8507818052594172
Total time per fold: 399.85715508461 seconds.
Fold : 5
Samples in training: 602
Samples in test: 67
Loaded pretrained weights for efficientnet-b0
		 Training: Epoch(0) - Loss: 2.1667, Acc: 52.1595
		 Validation(0) - Loss: 2.7846, Acc: 20.8955
		 Training: Epoch(1) - Loss: 0.5305, Acc: 86.5449
		 Validation(1) - Loss: 

		 Training: Epoch(2) - Loss: 0.2544, Acc: 93.3665
		 Validation(2) - Loss: 3.0639, Acc: 27.2727
		 Training: Epoch(3) - Loss: 0.2468, Acc: 91.8740
		 Validation(3) - Loss: 2.5022, Acc: 37.8788
		 Training: Epoch(4) - Loss: 0.1913, Acc: 95.5224
		 Validation(4) - Loss: 1.9315, Acc: 51.5152
		 Training: Epoch(5) - Loss: 0.0910, Acc: 97.5124
		 Validation(5) - Loss: 2.1657, Acc: 54.5455
		 Training: Epoch(6) - Loss: 0.0878, Acc: 97.0149
		 Validation(6) - Loss: 1.1792, Acc: 71.2121
		 Training: Epoch(7) - Loss: 0.0808, Acc: 97.8441
		 Validation(7) - Loss: 1.3743, Acc: 72.7273
		 Training: Epoch(8) - Loss: 0.0897, Acc: 98.0100
		 Validation(8) - Loss: 0.4742, Acc: 89.3939
		 Training: Epoch(9) - Loss: 0.0783, Acc: 98.0100
		 Validation(9) - Loss: 0.9297, Acc: 78.7879
		 Training: Epoch(10) - Loss: 0.1187, Acc: 96.8491
		 Validation(10) - Loss: 0.7763, Acc: 80.3030
		 Training: Epoch(11) - Loss: 0.1491, Acc: 95.5224
		 Validation(11) - Loss: 0.8020, Acc: 81.8182
		 Training: Epoch(12) - L

In [9]:
print("Train accuracy average: ", np.mean(train_acc) / 100)
print("Top-1 test accuracy average: ", np.mean(test_top1_acc))
print("Top-5 test accuracy average: ", np.mean(test_top5_acc))
print("Weighted Precision test accuracy average: ", np.mean(test_precision))
print("Weighted Recall test accuracy average: ", np.mean(test_recall))
print("Weighted F1 test accuracy average: ", np.mean(test_f1))
print("Average time per fold (seconds):", np.mean(times))

Train accuracy average:  0.928549684210913
Top-1 test accuracy average:  0.8938941655359566
Top-5 test accuracy average:  0.989507010402533
Weighted Precision test accuracy average:  0.9072431134371431
Weighted Recall test accuracy average:  0.8938941655359566
Weighted F1 test accuracy average:  0.8873449262933658
Average time per fold (seconds): 398.8333187818527
